In [19]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
from pytorch_lightning.utilities.seed import seed_everything
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning import Trainer
import wandb

seed_everything(42)
wandb.login()
wandb_logger = WandbLogger(project='neural-collaborative-filtering')

Global seed set to 42
wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


In [21]:
from ncf import NCF
from movielens_dataset import MovielensDataModule
data = MovielensDataModule()

data.setup()

trainer = Trainer(
    logger=wandb_logger,
    max_epochs=15
)


model = NCF(data.n_users(), data.n_movies())

GPU available: False, used: False
TPU available: None, using: 0 TPU cores


In [22]:
trainer.fit(model, data)


  | Name           | Type      | Params
---------------------------------------------
0 | user_embedding | Embedding | 151 K 
1 | item_embedding | Embedding | 97.1 K
2 | fc1            | Linear    | 5.1 K 
3 | fc2            | Linear    | 3.2 K 
4 | output         | Linear    | 33    
---------------------------------------------
256 K     Trainable params
0         Non-trainable params
256 K     Total params
1.026     Total estimated model params size (MB)
c:\users\szymon\projects\recommendation\sys-rek-l1\env\lib\site-packages\pytorch_lightning\utilities\distributed.py:51: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

1

In [23]:
import torch
torch.cuda.is_available()

False